In [1]:
from vrp.problem_instance import ProblemInstance
from vrp.vrp_solver import ExhaustiveHeuristic, NearestNeighborHeuristic, SweepAngleHeuristic, VRPSolution, VRPSolver

In [2]:
def get_opt_solution(problem_instance: ProblemInstance) -> VRPSolution:
    solver = VRPSolver(
        problem_instance,
        [ExhaustiveHeuristic()],
    )
    sol = solver.solve()
    return sol


def get_appx_solution(problem_instance: ProblemInstance) -> VRPSolution:
    solver = VRPSolver(
        problem_instance,
        [NearestNeighborHeuristic(), SweepAngleHeuristic()],
    )
    sol = solver.solve()
    return sol

In [3]:
# solve the example problem optimaly
problem_instance = ProblemInstance.from_csv(
    file_path="instances/example.csv",
    num_vehicles=3,
    vehciel_capacity=15,
)
opt_solution = get_opt_solution(problem_instance)
print(opt_solution)
opt_solution.visualize()

FileNotFoundError: [Errno 2] No such file or directory: 'instances/quince.csv'

In [4]:
# solve the example problem approximately
problem_instance = ProblemInstance.from_csv(
    file_path="instances/quince.csv",
    num_vehicles=3,
    vehciel_capacity=15,
)
appx_solution = get_appx_solution(problem_instance)
print(appx_solution)
appx_solution.visualize()


Path 0: Depot -> 1 -> 7 -> 4 -> 2

Path 1: Depot -> 6 -> 8 -> 9

Path 2: Depot -> 3 -> 5 -> 10

Total Cost: 62.64813890655621



In [5]:
import pandas as pd
import plotly.express as px


# profile optimality performance
def get_optimality_gap(problem_instance: ProblemInstance):
    opt_solution = get_opt_solution(problem_instance)
    appx_solution = get_appx_solution(problem_instance)
    return (opt_solution.distance - appx_solution.distance) / opt_solution.distance


optimality_gaps = [(i, get_optimality_gap(ProblemInstance.from_random(num_locations=i))) for i in range(10, 23, 4)]
optimality_gaps_frame = pd.DataFrame(optimality_gaps, columns=["num_locations", "optimality_gap"])

fig = px.line(optimality_gaps_frame, x="num_locations", y="optimality_gap")
fig

KeyboardInterrupt: 

In [6]:
import timeit


def get_runtime(problem_instance: ProblemInstance) -> float:
    start = timeit.default_timer()
    solver = VRPSolver(
        problem_instance,
        [NearestNeighborHeuristic(), SweepAngleHeuristic()],
    )
    sol = solver.solve()
    stop = timeit.default_timer()
    return stop - start

In [11]:
list(range(10, 23, 4))

[10, 14, 18, 22]